In [1]:
from pprint import pprint
from pathlib import Path
import pandas as pd
import qlib
import os
from qlib.utils import init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord, SigAnaRecord
from qlib.contrib.report import analysis_model, analysis_position
from qlib.data import D
from qlib.constant import REG_US

qlib.init(provider_uri='~/.qlib/qlib_data/us_data', region=REG_US)

/home/linq/anaconda3/envs/wk39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[1830615:MainThread](2024-11-02 15:14:47,579) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
[1830615:MainThread](2024-11-02 15:14:47,583) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[1830615:MainThread](2024-11-02 15:14:47,585) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/home/linq/.qlib/qlib_data/us_data')}


In [12]:
pos_df = pd.read_csv('/home/linq/finance/data/us_position_ref.csv')
pos_df.columns = ['trade_dt', 'dir', 'ticker', 'name', 'weight']
pos_df['ticker'] = pos_df['ticker'].apply(lambda s: s.split('.')[0])
pos_df['trade_dt'] = pd.to_datetime(pos_df['trade_dt'], format='%Y%m%d')
pos_df = pos_df.set_index(['trade_dt', 'ticker'])
pos_df['dir'] = pos_df['dir'].apply(lambda s: 1 if s=='多头' else -1)
pos_df['weight'] = pos_df['dir'] * pos_df['weight']
pos_df

dir                         name  weight
trade_dt   ticker                                          
2024-01-05 ADBE      1                   奥多比(ADOBE)  0.0133
           AJG       1  亚瑟加拉格尔(ARTHUR J. GALLAGHER)  0.0073
           ANET      1                     ARISTA网络  0.0100
           BIIB      1                 生物基因(BIOGEN)  0.0028
           BURL      1     伯灵顿百货(BURLINGTON STORES)  0.0129
...                ...                          ...     ...
2024-06-28 VAL       1                      VALARIS  0.0025
           VRT       1                       VERTIV  0.0084
           VRTX      1                 福泰制药(VERTEX)  0.0041
           VST       1                       VISTRA  0.0054
           WM        1     美国废物管理(WASTE MANAGEMENT)  0.0104

[1813 rows x 3 columns]

In [10]:
for _, r in pos_df[pos_df.index.get_level_values('trade_dt')=='2024-06-28'].iterrows():
    print(r['weight'])

0.0052
0.0043
0.0015
0.0024
0.0111
0.0103
0.0024
0.0361
0.001
0.0137
0.0034
0.0032
0.004
0.0025
0.0397
0.0029
0.0096
0.0096
0.0023
0.0023
0.022
0.0043
0.0162
0.0005
0.0041
0.0021
0.0224
0.0032
0.0012
0.0036
0.0012
0.0012
0.0313
0.0018
0.0011
0.0208
0.0018
0.0159
0.013
0.0033
0.0935
0.0141
0.1014
0.0026
0.0112
0.0243
0.0015
0.0264
0.0022
0.0011
0.0637
0.0018
0.0551
0.0231
0.0086
0.0031
0.0019
0.0017
0.0113
0.0016
0.0013
0.0074
0.0239
0.0032
0.0063
0.0019
0.0024
0.0057
0.0025
0.0049
0.003
0.0134
0.0043
0.002
0.016
0.0055
0.0025
0.0084
0.0041
0.0054
0.0104


In [14]:
pos_df.index.get_level_values('ticker').values

array(['ADBE', 'AJG', 'ANET', ..., 'VRTX', 'VST', 'WM'], dtype=object)